## Face mask detection

### Importing libraries 

In [7]:
import os
import cv2
import pathlib
import re
import glob

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import MobileNetV2

from sklearn.model_selection import train_test_split

### Global constants

In [27]:
data_dir = "../dataset"
batch_size = 32

data_dir = pathlib.Path(data_dir)
IMG_SIZE = 150

class_names = ['with_mask', 'without_mask']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

In [36]:
def prepare_dataset(path):
    """
    Read images from the path & preprocess it
    
    Args:
        path(String/pathlib) - path of the image files
    
    Returns:
        x - processed X data
        y - corresponding y labels of x
    """
    x = []
    y = []
    
    path = pathlib.Path(path)
    
    sub_folders = [x for x in data_dir.iterdir()] 
    
    for folder in sub_folders:
        images_path= folder.glob('**/*.jpg')
        
        for img_path in images_path :
            img=load_img(img_path, target_size=(IMG_SIZE,IMG_SIZE))
            img=img_to_array(img)
            img=img/255.0
            x.append(img)
            y.append(class_names_label[folder.name])
    
    return np.array(x),np.array(y)

In [37]:
x,y = prepare_dataset(data_dir)

In [38]:
def split_train_test(x,y,test_ratio = 0.2):
    """
    Split the given data into test and train set
    
    Args:
        x(numpy array): x data with first dimension as the samples
        y(numpy array): corresponding labels to x 
        test_ratio(0-1): ratio of data required for test set
        
    Returns:
        x_train(numpy_array) - training x data
        y_train(numpy_array) - training y data
        x_test(numpy_array) - test x data
        y_test(numpy_array) - test y data
        
    """
    
    X_train, X_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=test_ratio,
                                                    random_state=42)
    return X_train, X_test, y_train, y_test

In [39]:
x_train, x_test, y_train, y_test = split_train_test(x,y)

### Checking the shape of the content


In [41]:
print("Number of samples in training:" + str(x_train.shape[0]))
print("Number of samples in test:" + str(x_train.shape[1]))

print("Dimension of each sample: " + str(x_train.shape[1:]))

Number of samples in training:1100
Number of samples in test:150
Dimension of each sample: (150, 150, 3)


### Todo

* Display some random images
* Split the data to train and test set - done
* Display the class level split in both test and train

In [42]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (IMG_SIZE, IMG_SIZE, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [43]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 41472)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               5308544   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [44]:
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

In [ ]:
history=model.fit(x=x_train,y=y_train,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100
35/35 [==============================] - 11s 310ms/step - loss: 0.6183 - acc: 0.6600 - val_loss: 0.4551 - val_acc: 0.7754
Epoch 2/100
35/35 [==============================] - 10s 298ms/step - loss: 0.3530 - acc: 0.8782 - val_loss: 0.3800 - val_acc: 0.8442
Epoch 3/100
30/35 [========================>.....] - ETA: 1s - loss: 0.2577 - acc: 0.9135